In [20]:
import requests
import xml.etree.ElementTree as ET

# 1) 요청 (format="xml" 로 가정)
API_URL = "http://api.vworld.kr/ned/data/getIndvdLandPriceAttr"
params = {
    "key":       "6A1C6875-2F44-30B6-8E82-EDA8A02F0AF4",
    "domain":    "https://www.gujin.com",
    "pnu":       "1111017700102110000",
    "stdrYear":  "2015",
    "format":    "xml",
    "numOfRows": 10,
    "pageNo":    1
}
resp = requests.get(API_URL, params=params)
resp.raise_for_status()

# 2) XML 파싱
root = ET.fromstring(resp.text)
field = root.find(".//field")
if field is None:
    print("조회된 데이터가 없습니다.")
    exit()

# 3) 태그값 가져오기 함수
def get(tag):
    el = field.find(tag)
    return el.text if el is not None else ""

pnu         = get("pnu")
ldCode      = get("ldCode")
ldCodeNm    = get("ldCodeNm")
stdrYear    = get("stdrYear")
stdrMt      = get("stdrMt")
price       = get("pblntfPclnd")
pubDate     = get("pblntfDe")
lastUpdated = get("lastUpdtDt")

# 4) 보기 좋게 출력
print("=== 개별공시지가 조회 결과 ===")
print(f"PNU (지번고유번호):    {pnu}")
print(f"행정구역 코드:        {ldCode}")
print(f"행정구역 명칭:        {ldCodeNm}")
print(f"기준년도·월:          {stdrYear}-{stdrMt}")
print(f"공시지가(㎡ 당):      {int(price):,} 원")
print(f"공시(공고)일자:       {pubDate}")
print(f"최종갱신일자:         {lastUpdated}")


=== 개별공시지가 조회 결과 ===
PNU (지번고유번호):    1111017700102110000
행정구역 코드:        1111017700
행정구역 명칭:        서울특별시 종로구 평동
기준년도·월:          2015-01
공시지가(㎡ 당):      3,026,000 원
공시(공고)일자:       2015-05-29
최종갱신일자:         2023-08-10


In [24]:
import requests
import xml.etree.ElementTree as ET

# ─── 0) 공통 설정 ───────────────────────────────────────────────
API_KEY = "6A1C6875-2F44-30B6-8E82-EDA8A02F0AF4"
DOMAIN  = "https://www.gujin.com"   # 도메인제한 없으면 빈 문자열("")
# 관심 지점
lat, lon, delta = 37.571, 126.976, 0.001
bbox = f"{lon-delta},{lat-delta},{lon+delta},{lat+delta}"

# ─── 1) '지적도' 서비스 apiNum 조회 ─────────────────────────────
svc_list_url = "https://www.vworld.kr/dtna/dtna_apiSvcList_s001.do"
svc_params = {
    "key":          API_KEY,
    "domain":       DOMAIN,
    "format":       "xml",
    "searchKeyword":"지적도",
    "pageIndex":    1,
    "perPage":      100
}
resp = requests.get(svc_list_url, params=svc_params)
print("→ URL:", resp.url)
print("→ Status:", resp.status_code)
print("→ Content-Type:", resp.headers.get("Content-Type"))
print("→ Body Preview:", repr(resp.text[:200]))
resp.raise_for_status()
root = ET.fromstring(resp.text)

parcel_apiNum = None
# 실제 XML 구조에 맞게 'item' 혹은 'svc' 태그를 확인하세요.
for item in root.findall(".//item"):
    svcNm = item.findtext("svcNm")
    if svcNm and "지적" in svcNm:
        parcel_apiNum = item.findtext("apiNum")
        break

if not parcel_apiNum:
    raise RuntimeError("지적도 서비스(apiNum) 조회 실패")

# ─── 2) 지적도 WFS(FeatureCollection) 조회 → PNU 리스트 ────────────
wfs_url = "https://www.vworld.kr/dtna/dtna_apiSvcFc_s001.do"
wfs_params = {
    "key":    API_KEY,
    "domain": DOMAIN,
    "apiNum": parcel_apiNum,
    "format": "xml",
    "crs":    "EPSG:4326",
    "BBOX":   bbox
}
resp = requests.get(wfs_url, params=wfs_params)
resp.raise_for_status()
wfs_root = ET.fromstring(resp.text)

# XML 내 모든 <pnu> 태그를 찾아 텍스트 추출
pnus = [el.text for el in wfs_root.findall(".//pnu") if el.text]

if not pnus:
    print("해당 영역에 필지가 없습니다.")
    exit()

# ─── 3) 각 PNU별로 공시지가 조회 → 가격 리스트 수집 ────────────────
price_api = "http://api.vworld.kr/ned/data/getIndvdLandPriceAttr"
prices = []

for pnu in pnus:
    p_params = {
        "key":      API_KEY,
        "domain":   DOMAIN,
        "pnu":      pnu,
        "stdrYear": "2023",
        "format":   "xml"
    }
    r = requests.get(price_api, params=p_params)
    r.raise_for_status()
    doc = ET.fromstring(r.text)
    # 첫 번째 <field> 내부의 <pblntfPclnd> 값을 가져와 int 변환
    val = doc.findtext(".//pblntfPclnd")
    if val:
        prices.append(int(val))

# ─── 4) 평균 계산·출력 ────────────────────────────────────────────
if prices:
    avg = sum(prices) / len(prices)
    print(f"조회된 {len(prices)}개 필지의 평균 공시지가: {avg:,.0f} 원/㎡")
else:
    print("공시지가 데이터를 가져오지 못했습니다.")


→ URL: https://www.vworld.kr/dtna/dtna_apiSvcList_s001.do?key=6A1C6875-2F44-30B6-8E82-EDA8A02F0AF4&domain=https%3A%2F%2Fwww.gujin.com&format=xml&searchKeyword=%EC%A7%80%EC%A0%81%EB%8F%84&pageIndex=1&perPage=100
→ Status: 200
→ Content-Type: text/html;charset=utf-8
→ Body Preview: '<!doctype html>\r\n<html lang="ko">\r\n<head>\r\n\t<meta charset="utf-8">\r\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, viewp'


ParseError: syntax error: line 1, column 0 (<string>)